#### Duration: nearly instant

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
import logging
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/bin/python3


### Get recording info (google sheets)

In [3]:
import requests
import pandas as pd
from io import BytesIO

In [4]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [5]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams,max_video_duration_m
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10


### Run sync

In [6]:
for idx, recording_row in recording_df.iterrows():
    break

In [7]:
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240806-3d-pipeline"

In [13]:
output_directory_tensorrt = output_directory / 'tensorrt'
output_directory_tensorrt.mkdir(parents=True, exist_ok=True)

In [14]:
!ls /n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt/

rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661
rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524


In [15]:
print(output_directory_tensorrt)

/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt


In [16]:
from multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt import RTMModelConverter

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:Python interpreter binary location: /n/groups/datta/tim_sainburg/conda_envs/mmdeploy/bin/python3


In [17]:
# https://o2portal.rc.hms.harvard.edu/node/compute-g-17-148.o2.rc.hms.harvard.edu/59141/notebooks/tim_sainburg/projects/23-09-29-peromoseq/notebooks/keypoints/mmpose/24-01-11-deploy-2d-predictions-on-local/24-05-08-convert-pose-model.ipynb
rtmpose_model_name = 'rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524'
path_to_rmpose_config = Path('/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/config.py')
path_to_rmpose_checkpoint = Path('/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/best_PCK_epoch_200.pth')

In [18]:
# https://o2portal.rc.hms.harvard.edu/node/compute-g-17-148.o2.rc.hms.harvard.edu/59141/notebooks/tim_sainburg/projects/23-09-29-peromoseq/notebooks/keypoints/mmpose/24-01-11-deploy-2d-predictions-on-local/240508-convert-detection-model-fp32.ipynb
rtmdetection_model_name = 'rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661'
path_to_rtmdetection_config = Path('/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/config.py')
path_to_rtmdetection_checkpoint = Path('/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/epoch_400.pth')

In [19]:
model_converter = RTMModelConverter(
    path_to_rmpose_config = path_to_rmpose_config,
    path_to_rmpose_checkpoint = path_to_rmpose_checkpoint,
    path_to_rtmdetection_config = path_to_rtmdetection_config,
    path_to_rtmdetection_checkpoint = path_to_rtmdetection_checkpoint,
    tensorrt_output_directory = output_directory_tensorrt,
    rtmdetection_model_name = rtmdetection_model_name,
    rtmpose_model_name = rtmpose_model_name,
)

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:Using CUDA device: NVIDIA_L40S


In [20]:
model_converter.run()

08/13 09:04:07 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.
08/13 09:04:07 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "mmdet_tasks" registry tree. As a workaround, the current "mmdet_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.
08/13 09:04:11 - mmengine - INFO - Start pipeline mmdeploy.apis.pytorch2onnx.torch2onnx in subprocess
08/13 09:04:11 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry i

[08/13/2024-09:05:45] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +8, now: CPU 1926, GPU 1370 (MiB)
[08/13/2024-09:05:45] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +10, now: CPU 1926, GPU 1380 (MiB)
[08/13/2024-09:05:45] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in building engine: CPU +3, GPU +29, now: CPU 3, GPU 29 (MiB)
08/13 09:05:46 - mmengine - INFO - Finish pipeline mmdeploy.apis.utils.utils.to_backend
08/13 09:05:46 - mmengine - INFO - visualize tensorrt model start.
08/13 09:05:52 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.
08/13 09:05:52 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "mmdet_tasks" registry tree.

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:Finished converting detection model to tensorrt.
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:model output at: /n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/NVIDIA_L40S


08/13 09:06:08 - mmengine - WARNING - Failed to search registry with scope "mmpose" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpose" is a correct scope, or whether the registry is initialized.
08/13 09:06:08 - mmengine - WARNING - Failed to search registry with scope "mmpose" in the "mmpose_tasks" registry tree. As a workaround, the current "mmpose_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpose" is a correct scope, or whether the registry is initialized.
08/13 09:06:11 - mmengine - INFO - Start pipeline mmdeploy.apis.pytorch2onnx.torch2onnx in subprocess
08/13 09:06:12 - mmengine - WARNING - Failed to search registry with scope "mmpose" in the "Codebases" registry tree. As a workaround, the current "Codebases" reg

Loads checkpoint by local backend from path: /n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/best_PCK_epoch_200.pth
08/13 09:08:48 - mmengine - WARNING - render and display result skipped for headless device, exception no display name and no $DISPLAY environment variable
08/13 09:08:49 - mmengine - INFO - visualize pytorch model success.
08/13 09:08:49 - mmengine - INFO - All process success.


INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:Finished converting pose model to tensorrt.
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.tensorrt:model output at: /n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/NVIDIA_L40S


In [21]:
!ls {output_directory_tensorrt.as_posix()}

rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661
rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524


In [16]:
output_directory_tensorrt

PosixPath('/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt-2')

In [23]:
!ls {output_directory_tensorrt / "rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/NVIDIA_L40S"}

deploy.json  end2end.engine  output_pytorch.jpg   pipeline.json
detail.json  end2end.onnx    output_tensorrt.jpg


In [24]:
!ls {output_directory_tensorrt / "rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/NVIDIA_L40S"}

deploy.json  end2end.engine  output_pytorch.jpg   pipeline.json
detail.json  end2end.onnx    output_tensorrt.jpg
